In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os
import json

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
import os
if IN_COLAB:
    print("running in google colab")
    from google.colab import drive
    drive.mount("/content/drive")
    os.chdir("/content/drive/My Drive/seminar_dl_workspace")
    print("switched workspace:", os.getcwd())

from create_visualizations import get_filenames_in_dir

In [ ]:
from IPython.display import SVG, display
def show_svg(filename):
    display(SVG(filename=filename))

boilerplate done...

In [ ]:
def get_latest_experiment_df(csv_path):
    experiment_stats = pd.read_csv(csv_path)
    experiment_stats_grouped = experiment_stats.groupby("experiment_id")
    latest_training_session = experiment_stats_grouped.get_group(experiment_stats.experiment_id.max())
    if len(experiment_stats) < 20:
        print(f"WARNING: file {csv_path} has only {len(experiment_stats)} epochs but should have 20")
    elif len(experiment_stats) > 20 and len(experiment_stats) % 20 == 0:
        print(f"INFO: file {csv_path} has {experiment_stats_grouped.ngroups} training sessions. Using latest training session with {len(latest_training_session)} epochs and experiment_id {latest_training_session.iloc[0]['experiment_id']}")
    return latest_training_session

Find best results of the 20 epoch runs. A 100 epoch experiment will be run for each best param set. "Best" means highest test accuracy here.

In [ ]:
def strip_params_from_experiment_title(title):
    matches = re.search(r"^(\D+)_", title)
    return matches.group(1)

def get_best_row(df, col, best_value):
    return df[df[col] == best_value][["train_acc", "test_acc", "train_avg_loss", "test_avg_loss", "experiment_params", "experiment_id", "epoch"]].iloc[0]

experiments_stats_filenames = get_filenames_in_dir("experiments_stats", lambda x: "wip_cifar_resnet_sgd.csv" not in x)
print(f"found {len(experiments_stats_filenames)} total 20 epoch experiments")
df = pd.concat(map(lambda x: pd.read_csv(os.path.join("experiments_stats", x)), experiments_stats_filenames))
df["experiment_title"] = df["experiment_title"].apply(strip_params_from_experiment_title)
experiments = df.groupby("experiment_title")

def output_group_results(group_name, title):
    group = experiments.get_group(group_name)
    best_of_group = get_best_row(group, "test_acc", group["test_acc"].max())
    best_of_group_params = json.loads(best_of_group["experiment_params"])
    print(f"best {title}")
    for col in ["train_acc", "test_acc", "train_avg_loss", "test_avg_loss"]:
        print(f"  {col}: {round(best_of_group[col], 3)}")
    print(f"  params: {best_of_group_params['params']}")
    print("")

output_group_results("SGD_with_variable_batch_size", "vanilla mini-batch sgd")
output_group_results("SGD_with_momentum", "sgd with momentum")
output_group_results("Adagrad_batch_size", "adagrad")
output_group_results("RMSprop", "rmsprop")

## Results for 100 epoch experiments

In [ ]:
results_filenames = get_filenames_in_dir("experiments_stats/100_epochs")
results = pd.concat(map(lambda x: pd.read_csv(os.path.join("experiments_stats/100_epochs", x)), results_filenames))
results["experiment_title"] = results["experiment_title"].apply(strip_params_from_experiment_title)
results = results.groupby("experiment_title")

In [ ]:
results.agg({"train_acc": np.max, "test_acc": np.max, "train_avg_loss": np.min, "test_avg_loss": np.min})

Plots for 100 epoch experiments

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(18, 15), gridspec_kw=dict(hspace=0.35))
fig.suptitle("Accuracy and loss for 100 epochs experiments")
top = axs[0]
bot = axs[1]
top.margins(x=0.01)
bot.margins(x=0.01)

labels=[
    "train SGD",
    "test SGD",
    "train SGD+Momentum",
    "test SGD+Momentum",
    "train SGD+Momentum+LR_Schedule",
    "test SGD+Momentum+LR_Schedule",
    "train Adagrad",
    "test Adagrad",
    "train RMSprop",
    "test RMSprop",
    ]
acc_cols = ["train_acc", "test_acc"]
loss_cols = ["train_avg_loss", "test_avg_loss"]

def plot_group(ax, group, cols, title, ylabel):
    line_objects = ax.plot(group[cols])
    ax.set_title(title)
    ax.set_xlabel("epochs")
    ax.set_ylabel(ylabel)
    return line_objects

acc_lines = []
loss_lines = []
group_names = list(results.groups.keys())
adagrad = group_names.pop(0)
rmsprop = group_names.pop(0)
group_names.append(adagrad)
group_names.append(rmsprop)

# plot results of n experiment groups. Used to create incremental visualization
group_num = 4
for group_name in group_names[:group_num + 1]:
    group = results.get_group(group_name)
    acc_lines += plot_group(top, group, acc_cols, title="Train and Test Accuracy", ylabel="accuracy")
    loss_lines += plot_group(bot, group, loss_cols, title="Train and Test Loss", ylabel="loss")
fig.legend(iter(acc_lines), labels, loc="upper left", borderaxespad=0.1, title="Legend")

# make newest lines fat
for line in acc_lines[-2:] + loss_lines[-2:]:
    line.set_linewidth(3.0)
    line.zorder = 99

plt.savefig(f"visualizations/final_100_epochs_{group_num}.png")

results for a longer 896 epochs training of sgd + momentum + lr schedule cosine annealling

The google colab time ran out in epoch 897, but training wasn't continued since it looks like further training wouldn't have improved anyhting

In [ ]:
long_experiment = pd.read_csv("experiments_stats/dont_use/SGD_with_momentum_and_lr_scheduled_larger_1000_epochs_batch_size_16_lr_0.04_momentum_0.9.csv")
print("train acc:", round(long_experiment["train_acc"].max(), 4))
print("test acc:", round(long_experiment["test_acc"].max(), 4))
print("train loss:", round(long_experiment["train_avg_loss"].min(), 4))
print("test loss:", round(long_experiment["test_avg_loss"].min(), 4))
fig, axs = plt.subplots(2, 1, figsize=(18, 15), gridspec_kw=dict(hspace=0.35))
fig.suptitle("Accuracy and loss for 896 epochs of SGD + Momentum + LR-Schedule Cosine Annealling")
top = axs[0]
bot = axs[1]
top.margins(x=0.01)
bot.margins(x=0.01)
acc_lines, loss_lines = [], []
acc_lines += plot_group(top, long_experiment, acc_cols, title="Train and Test Accuracy", ylabel="accuracy")
loss_lines += plot_group(bot, long_experiment, loss_cols, title="Train and Test Loss", ylabel="loss")
fig.legend(iter(acc_lines), ["train", "test"], loc="upper left", borderaxespad=0.1, title="Legend")
plt.savefig("visualizations/sgd_momentum_lr_scheduled_896_epochs.png")

plot learning rates from cosine annealling with different parameters T_0 and T_mult

In [ ]:
cosine_annealling_lrs_b = [0.04, 0.03801937735804838, 0.032469796037174675, 0.024450418679126288, 0.015549581320873713, 0.007530203962825331, 0.0019806226419516195, 0.04, 0.03949855824363647, 0.03801937735804838, 0.035636629649360596, 0.032469796037174675, 0.028677674782351163, 0.024450418679126288, 0.02, 0.015549581320873713, 0.01132232521764884, 0.007530203962825331, 0.004363370350639411,
0.0019806226419516195, 0.0005014417563635255, 0.04, 0.03987424419786485, 0.03949855824363647, 0.03887766660616735, 0.03801937735804838, 0.03693448398456568, 0.035636629649360596, 0.03414213562373095, 0.032469796037174675, 0.030640641530306732, 0.028677674782351163, 0.026605581239103344, 0.024450418679126288, 0.022239289522066153, 0.02, 0.017760710477933848, 0.015549581320873713, 0.013394418760896656, 0.01132232521764884, 0.00935935846969327, 0.007530203962825331, 0.005857864376269051, 0.004363370350639411, 0.003065516015434318, 0.0019806226419516195, 0.0011223333938326484, 0.0005014417563635255, 0.00012575580213514793, 0.04, 0.039968536300356336, 0.03987424419786485, 0.039717420370364714, 0.03949855824363647, 0.03921834643890199, 0.03887766660616735, 0.03847759065022573, 0.03801937735804838, 0.03750446843817508, 0.03693448398456568, 0.036311217379185204, 0.035636629649360596, 0.03491284329766331, 0.03414213562373095, 0.03332693155904008, 0.032469796037174675, 0.03157342592359612, 0.030640641530306732, 0.029674377742104796, 0.028677674782351163, 0.0276536686473018, 0.026605581239103344, 0.025536710228496986, 0.024450418679126288, 0.02335012446609473, 0.022239289522066153, 0.021121408944743836, 0.02, 0.018878591055256168, 0.017760710477933848, 0.016649875533905276, 0.015549581320873713, 0.014463289771503015, 0.013394418760896656, 0.012346331352698206, 0.01132232521764884, 0.010325622257895205, 0.00935935846969327, 0.008426574076403886, 0.007530203962825331, 0.006673068440959922, 0.005857864376269051, 0.005087156702336689, 0.004363370350639411, 0.003688782620814797, 0.003065516015434318, 0.0024955315618249265, 0.0019806226419516195, 0.0015224093497742653, 0.0011223333938326484]
plt.plot(cosine_annealling_lrs_b)
plt.title("Learning rate per epoch using cosine annealling\nwith T_0=7 and T_mult=2 for 100 epochs")
plt.xlabel("epoch")
plt.ylabel("learning rate")
plt.savefig("visualizations/cosine_annealling_learning_rate_100_epochs_T0-7_Tmult-2.png")

In [ ]:
cosine_annealling_lrs_c = [0.04, 0.03801937735804838, 0.032469796037174675, 0.024450418679126288, 0.015549581320873713, 0.007530203962825331, 0.0019806226419516195, 0.04, 0.03949855824363647, 0.03801937735804838, 0.035636629649360596, 0.032469796037174675, 0.028677674782351163, 0.024450418679126288, 0.02, 0.015549581320873713, 0.01132232521764884, 0.007530203962825331, 0.004363370350639411,
0.0019806226419516195, 0.0005014417563635255, 0.04, 0.03987424419786485, 0.03949855824363647, 0.03887766660616735, 0.03801937735804838, 0.03693448398456568, 0.035636629649360596, 0.03414213562373095, 0.032469796037174675, 0.030640641530306732, 0.028677674782351163, 0.026605581239103344, 0.024450418679126288, 0.022239289522066153, 0.02, 0.017760710477933848, 0.015549581320873713, 0.013394418760896656, 0.01132232521764884, 0.00935935846969327, 0.007530203962825331, 0.005857864376269051, 0.004363370350639411, 0.003065516015434318, 0.0019806226419516195, 0.0011223333938326484, 0.0005014417563635255, 0.00012575580213514793, 0.04, 0.039968536300356336, 0.03987424419786485, 0.039717420370364714, 0.03949855824363647, 0.03921834643890199, 0.03887766660616735, 0.03847759065022573, 0.03801937735804838, 0.03750446843817508, 0.03693448398456568, 0.036311217379185204, 0.035636629649360596, 0.03491284329766331, 0.03414213562373095, 0.03332693155904008, 0.032469796037174675, 0.03157342592359612, 0.030640641530306732, 0.029674377742104796, 0.028677674782351163, 0.0276536686473018, 0.026605581239103344, 0.025536710228496986, 0.024450418679126288, 0.02335012446609473, 0.022239289522066153, 0.021121408944743836, 0.02, 0.018878591055256168, 0.017760710477933848, 0.016649875533905276, 0.015549581320873713, 0.014463289771503015, 0.013394418760896656, 0.012346331352698206, 0.01132232521764884, 0.010325622257895205, 0.00935935846969327, 0.008426574076403886, 0.007530203962825331, 0.006673068440959922, 0.005857864376269051, 0.005087156702336689, 0.004363370350639411, 0.003688782620814797, 0.003065516015434318, 0.0024955315618249265, 0.0019806226419516195, 0.0015224093497742653, 0.0011223333938326484, 0.0007816535610980103, 0.0005014417563635255, 0.00028257962963528226, 0.00012575580213514793, 3.146369964366791e-05, 0.04, 0.03999213252766106, 0.039968536300356336, 0.03992922988235239, 0.03987424419786485, 0.03980362250672891, 0.039717420370364714, 0.03961570560806461, 0.03949855824363647, 0.03936607044244523, 0.03921834643890199, 0.03905550245445793, 0.03887766660616735, 0.038684978805891994, 0.03847759065022573, 0.03825566530122638, 0.03801937735804838, 0.03776891271957744, 0.03750446843817508,
0.037226252564648184, 0.03693448398456568, 0.036629392246050906, 0.036311217379185204, 0.035980209707164985, 0.035636629649360596, 0.03528074751643215, 0.03491284329766331, 0.034533206440680544, 0.03414213562373095, 0.03373993852069804, 0.03332693155904008, 0.03290343967084176, 0.032469796037174675, 0.03202634182596812, 0.03157342592359612, 0.031111404660392045, 0.030640641530306732, 0.03016150690493059, 0.029674377742104796, 0.029179637289350753, 0.028677674782351163, 0.028168885138719926, 0.0276536686473018, 0.02713243065324626, 0.026605581239103344, 0.026073534902192295, 0.025536710228496986, 0.02499552956334454, 0.024450418679126288, 0.023901806440322568, 0.02335012446609473, 0.022795806790709992, 0.022239289522066153, 0.02168101049858495, 0.021121408944743836, 0.02056092512551738, 0.02, 0.019439074874482622, 0.018878591055256168, 0.018318989501415053, 0.017760710477933848, 0.017204193209290012, 0.016649875533905276, 0.016098193559677437, 0.015549581320873713, 0.015004470436655464, 0.014463289771503015, 0.013926465097807705, 0.013394418760896656, 0.012867569346753741, 0.012346331352698206, 0.011831114861280079, 0.01132232521764884, 0.010820362710649251, 0.010325622257895205, 0.009838493095069416, 0.00935935846969327, 0.008888595339607954, 0.008426574076403886, 0.00797365817403188, 0.007530203962825331, 0.007096560329158253, 0.006673068440959922, 0.006260061479301966, 0.005857864376269051, 0.005466793559319458, 0.005087156702336689, 0.004719252483567855, 0.004363370350639411, 0.004019790292835017, 0.003688782620814797, 0.0033706077539490996, 0.003065516015434318, 0.0027737474353518254, 0.0024955315618249265, 0.0022310872804225525, 0.0019806226419516195, 0.0017443346987736242, 0.0015224093497742653, 0.0013150211941080016, 0.0011223333938326484, 0.0009444975455420734, 0.0007816535610980103, 0.0006339295575547732, 0.0005014417563635255, 0.0003842943919353914, 0.00028257962963528226, 0.00019637749327108933, 0.00012575580213514793, 7.077011764761832e-05, 3.146369964366791e-05, 7.867472338942694e-06, 0.04, 0.039998033035191505, 0.03999213252766106, 0.03998229963801774, 0.039968536300356336, 0.03995084522187692, 0.03992922988235239, 0.03990369453344394, 0.03987424419786485, 0.039840884668392494, 0.03980362250672891, 0.03976246504221018, 0.039717420370364714, 0.03966849735132097, 0.03961570560806461, 0.03955905552454576, 0.03949855824363647, 0.039434225664939024, 0.03936607044244523, 0.039294105982047516, 0.03921834643890199, 0.039138806714644175, 0.03905550245445793, 0.03896845004399806, 0.03887766660616735, 0.03878316999774856, 0.038684978805891994, 0.03858311234445957, 0.03847759065022573, 0.038368434478936425, 0.03825566530122638, 0.03813930529839601, 0.03801937735804838, 0.03789590506958732, 0.03776891271957744, 0.0376384252869671, 0.03750446843817508, 0.037367068522042114, 0.037226252564648184, 0.037082048263996546, 0.03693448398456568, 0.036783588751730045, 0.036629392246050906, 0.0364719247974383, 0.036311217379185204, 0.03614730160187525, 0.035980209707164985, 0.03580997456144207, 0.035636629649360596, 0.035460209067254736, 0.03528074751643215, 0.03509828029634832, 0.03491284329766331, 0.03472447299518219, 0.034533206440680544, 0.03433908125561655, 0.03414213562373095, 0.033942408283536465, 0.03373993852069804, 0.033534766160305485, 0.03332693155904008, 0.03311647559723648, 0.03290343967084176, 0.03268786568327291, 0.032469796037174675, 0.032249273626079, 0.03202634182596812, 0.03180104448674256, 0.03157342592359612, 0.03134353090829912, 0.031111404660392045, 0.030877092838290973, 0.030640641530306732, 0.030402097245579497, 0.03016150690493059, 0.02991891783163336, 0.029674377742104796, 0.02942793473651995, 0.029179637289350753, 0.02892953423983126, 0.028677674782351163, 0.02842410845677942, 0.028168885138719926, 0.027912055029701177, 0.0276536686473018, 0.027393776815213898, 0.02713243065324626, 0.026869681567269224, 0.026605581239103344, 0.02634018161635376, 0.026073534902192295, 0.02580569354508925, 0.025536710228496986, 0.0252666378604873, 0.02499552956334454, 0.024723438663116687, 0.024450418679126288, 0.024176523313443398, 0.023901806440322568, 0.023626322095605962, 0.02335012446609473, 0.0230732678788906, 0.022795806790709992, 0.02251779577717251, 0.022239289522066153, 0.021960342806591208, 0.02168101049858495, 0.02140134754172935, 0.021121408944743836, 0.020841249770565305, 0.02056092512551738, 0.020280490148471247, 0.02,
0.019719509851528757, 0.019439074874482622, 0.0191587502294347, 0.018878591055256168, 0.018598652458270654, 0.018318989501415053, 0.018039657193408793, 0.017760710477933848, 0.01748220422282749, 0.017204193209290012, 0.0169267321211094, 0.016649875533905276, 0.01637367790439404, 0.016098193559677437, 0.015823476686556603, 0.015549581320873713, 0.015276561336883319,
0.015004470436655464, 0.014733362139512703, 0.014463289771503015, 0.014194306454910752, 0.013926465097807705, 0.01365981838364624, 0.013394418760896656, 0.013130318432730777, 0.012867569346753741, 0.012606223184786103, 0.012346331352698206, 0.012087944970298827, 0.011831114861280079, 0.011575891543220584, 0.01132232521764884, 0.011070465760168744, 0.010820362710649251, 0.010572065263480047, 0.010325622257895205, 0.01008108216836665, 0.009838493095069416, 0.009597902754420507, 0.00935935846969327, 0.009122907161709031, 0.008888595339607954, 0.008656469091700883, 0.008426574076403886, 0.00819895551325744, 0.00797365817403188, 0.007750726373920998, 0.007530203962825331, 0.007312134316727093, 0.007096560329158253, 0.006883524402763526, 0.006673068440959922, 0.006465233839694518, 0.006260061479301966, 0.006057591716463535, 0.005857864376269051, 0.005660918744383457, 0.005466793559319458, 0.005275527004817813, 0.005087156702336689, 0.0049017197036516835, 0.004719252483567855, 0.004539790932745267, 0.004363370350639411, 0.004190025438557927, 0.004019790292835017, 0.0038526983981247522, 0.003688782620814797, 0.003528075202561707, 0.0033706077539490996, 0.003216411248269957, 0.003065516015434318, 0.0029179517360034524, 0.0027737474353518254, 0.002632931477957892, 0.0024955315618249265, 0.002361574713032897, 0.0022310872804225525, 0.0021040949304126787, 0.0019806226419516195, 0.0018606947016039932, 0.0017443346987736242, 0.0016315655210635827, 0.0015224093497742653, 0.0014168876555404376, 0.0013150211941080016, 0.0012168300022514433, 0.0011223333938326484, 0.001031549956001947, 0.0009444975455420734, 0.0008611932853558235, 0.0007816535610980103, 0.0007058940179524842, 0.0006339295575547732, 0.0005657743350609823, 0.0005014417563635255, 0.00044094447545424577, 0.0003842943919353914, 0.0003315026486790318, 0.00028257962963528226, 0.00023753495778982447, 0.00019637749327108933, 0.00015911533160750668, 0.00012575580213514793, 9.630546655606365e-05, 7.077011764761832e-05, 4.915477812307945e-05, 3.146369964366791e-05, 1.7700361982262436e-05, 7.867472338942694e-06, 1.9669648084996717e-06, 0.04, 0.039999508252752496, 0.039998033035191505, 0.039995574419860444, 0.03999213252766106, 0.039987707527847444, 0.03998229963801774, 0.03997590912410345, 0.039968536300356336, 0.039960181529332964, 0.03995084522187692, 0.03994052783709855, 0.03992922988235239, 0.03991695191321226, 0.03990369453344394, 0.0398894583949754, 0.03987424419786485, 0.039858052690266245, 0.039840884668392494, 0.03982274097647637, 0.03980362250672891, 0.039783530199295626, 0.03976246504221018, 0.03974042807134588, 0.039717420370364714, 0.03969344307066404, 0.03966849735132097, 0.03964258443903438, 0.03961570560806461, 0.03958786218017076, 0.03955905552454576, 0.039529287057748934, 0.03949855824363647, 0.039466870593289335, 0.039434225664939024, 0.03940062506389088, 0.03936607044244523, 0.03933056349981606, 0.039294105982047516, 0.03925669968192799, 0.03921834643890199, 0.0391790481389797, 0.039138806714644175, 0.03909762414475641, 0.03905550245445793, 0.03901244371507126, 0.03896845004399806, 0.038923523604614985, 0.03887766660616735, 0.038830881303660414, 0.03878316999774856, 0.03873453503462211, 0.038684978805891994, 0.03863450374847213, 0.03858311234445957, 0.03853080712101244, 0.03847759065022573, 0.03842346554900474, 0.038368434478936425, 0.038312500146158494, 0.03825566530122638, 0.03819793273897793, 0.03813930529839601, 0.03807978586246887, 0.03801937735804838, 0.037958082755706134, 0.03789590506958732, 0.03783284735726254, 0.03776891271957744, 0.037704104300500245, 0.0376384252869671, 0.03757187890872543, 0.03750446843817508, 0.037436197190207354, 0.037367068522042114, 0.037297085833062564, 0.037226252564648184, 0.03715457220000544, 0.037082048263996546, 0.0370086843229661, 0.03693448398456568, 0.03685945089757653, 0.036783588751730045, 0.036706901277526356, 0.036629392246050906, 0.03655106546878898, 0.0364719247974383, 0.036391974123719584, 0.036311217379185204, 0.036229658535025855, 0.03614730160187525, 0.0360641506296129, 0.035980209707164985, 0.03589548296230326, 0.03580997456144207, 0.035723688709433515, 0.035636629649360596, 0.03554880166232865, 0.035460209067254736, 0.03537085622065536, 0.03528074751643215, 0.03518988738565583, 0.03509828029634832, 0.035005930753263026, 0.03491284329766331, 0.03481902250709919, 0.03472447299518219, 0.03462919941135856, 0.034533206440680544,
0.03443649880357606, 0.03433908125561655, 0.03424095858728311, 0.03414213562373095, 0.03404261722455211, 0.033942408283536465, 0.03384151372843111, 0.03373993852069804, 0.0336376876552701, 0.033534766160305485, 0.03343117909694037, 0.03332693155904008, 0.0332220286729486, 0.03311647559723648, 0.033010277522447166, 0.03290343967084176, 0.03279596729614219, 0.03268786568327291, 0.03257914014810098, 0.032469796037174675, 0.03235983872746053, 0.032249273626079, 0.03213810617003851, 0.03202634182596812, 0.03191398608984867, 0.03180104448674256, 0.031687522570522056, 0.03157342592359612, 0.03145876015663596, 0.03134353090829912, 0.03122774384495216, 0.031111404660392045, 0.030994519075566166, 0.030877092838290973, 0.03075913172296937, 0.030640641530306732, 0.030521628087025673, 0.030402097245579497, 0.030282054883864432, 0.03016150690493059, 0.03004045923669164, 0.02991891783163336, 0.02979688866652087, 0.029674377742104796, 0.02955139108282612, 0.02942793473651995, 0.02930401477411815, 0.029179637289350753, 0.029054808398446343, 0.02892953423983126, 0.028803820973827783, 0.028677674782351163, 0.02855110186860564, 0.02842410845677942, 0.02829670079173856, 0.028168885138719926, 0.028040667783023072, 0.027912055029701177, 0.02778305320325098, 0.0276536686473018, 0.027523907724303544, 0.027393776815213898, 0.027263282319184495, 0.02713243065324626, 0.027001228251993848, 0.026869681567269224, 0.026737797067844404, 0.026605581239103344, 0.026473040582723026,
0.02634018161635376, 0.026207010873298638, 0.026073534902192295, 0.025939760266678873, 0.02580569354508925, 0.02567134133011756, 0.025536710228496986, 0.025401806860674912, 0.0252666378604873, 0.02513120987483253, 0.02499552956334454, 0.024859603598065275, 0.024723438663116687, 0.024587041454371957, 0.024450418679126288, 0.02431357705576705, 0.024176523313443398, 0.024039264191735396, 0.023901806440322568, 0.023764156818652005, 0.023626322095605962, 0.023488309049169015, 0.02335012446609473, 0.023211775141571942, 0.0230732678788906, 0.022934609489107236, 0.022795806790709992, 0.022656866609283358, 0.02251779577717251, 0.022378601133147345, 0.022239289522066153, 0.02209986779453909, 0.021960342806591208, 0.021820721419325402, 0.02168101049858495, 0.02154121691461593, 0.02140134754172935, 0.02126140925796312, 0.021121408944743836, 0.020981353486548362, 0.020841249770565305, 0.020701104686356327, 0.02056092512551738, 0.020420717981339775, 0.020280490148471247, 0.02014024852257687, 0.02, 0.01985975147742313, 0.019719509851528757, 0.019579282018660226, 0.019439074874482622, 0.019298895313643677, 0.0191587502294347, 0.01901864651345164, 0.018878591055256168, 0.018738590742036883, 0.018598652458270654, 0.01845878308538407, 0.018318989501415053, 0.0181792785806746, 0.018039657193408793, 0.017900132205460915, 0.017760710477933848, 0.01762139886685266, 0.01748220422282749, 0.017343133390716647, 0.017204193209290012, 0.017065390510892765, 0.0169267321211094,
0.016788224858428062, 0.016649875533905276, 0.016511690950830986, 0.01637367790439404, 0.016235843181348, 0.016098193559677437, 0.015960735808264605, 0.015823476686556603, 0.01568642294423295, 0.015549581320873713, 0.015412958545628047, 0.015276561336883319, 0.015140396401934729, 0.015004470436655464, 0.014868790125167468, 0.014733362139512703, 0.014598193139325094, 0.014463289771503015, 0.01432865866988245, 0.014194306454910752, 0.01406023973332113, 0.013926465097807705, 0.013792989126701365, 0.01365981838364624, 0.013526959417276973, 0.013394418760896656, 0.013262202932155596, 0.013130318432730777, 0.012998771748006153, 0.012867569346753741, 0.012736717680815506, 0.012606223184786103, 0.012476092275696456, 0.012346331352698206, 0.01221694679674902, 0.012087944970298827, 0.011959332216976927, 0.011831114861280079, 0.011703299208261441, 0.011575891543220584, 0.011448898131394363, 0.01132232521764884, 0.011196179026172216, 0.011070465760168744, 0.010945191601553661, 0.010820362710649251, 0.010695985225881853, 0.010572065263480047, 0.010448608917173878, 0.010325622257895205, 0.010203111333479127, 0.01008108216836665, 0.009959540763308362, 0.009838493095069416, 0.009717945116135567, 0.009597902754420507, 0.009478371912974338, 0.00935935846969327, 0.00924086827703063, 0.009122907161709031, 0.009005480924433838, 0.008888595339607954, 0.008772256155047847, 0.008656469091700883, 0.008541239843364044, 0.008426574076403886, 0.008312477429477954, 0.00819895551325744, 0.008086013910151332, 0.00797365817403188, 0.007861893829961488, 0.007750726373920998, 0.007640161272539474, 0.007530203962825331, 0.007420859851899016, 0.007312134316727093, 0.007204032703857815, 0.007096560329158253, 0.006989722477552835, 0.006883524402763526, 0.006777971327051398, 0.006673068440959922, 0.006568820903059631, 0.006465233839694518, 0.006362312344729899, 0.006260061479301966, 0.006158486271568888, 0.006057591716463535, 0.0059573827754478975, 0.005857864376269051, 0.005759041412716899, 0.005660918744383457, 0.005563501196423948, 0.005466793559319458, 0.005370800588641449, 0.005275527004817813, 0.0051809774929008155, 0.005087156702336689, 0.004994069246736972, 0.0049017197036516835, 0.004810112614344173, 0.004719252483567855, 0.00462914377934464, 0.004539790932745267, 0.0044511983376713605, 0.004363370350639411, 0.004276311290566488, 0.004190025438557927, 0.004104517037696749, 0.004019790292835017, 0.003935849370387104, 0.0038526983981247522, 0.0037703414649741473, 0.003688782620814797, 0.0036080258762804275, 0.003528075202561707, 0.0034489345312110208, 0.0033706077539490996, 0.003293098722473642, 0.003216411248269957, 0.0031405491024234645, 0.003065516015434318, 0.0029913156770339055, 0.0029179517360034524, 0.00284542779999456, 0.0027737474353518254, 0.0027029141669374403, 0.002632931477957892, 0.0025638028097926457, 0.0024955315618249265, 0.002428121091274571, 0.002361574713032897, 0.0022958956994997594, 0.0022310872804225525, 0.00216715264273746, 0.0021040949304126787, 0.002041917244293867, 0.0019806226419516195, 0.0019202141375311333, 0.0018606947016039932, 0.001802067261022069, 0.0017443346987736242, 0.0016874998538415076, 0.0016315655210635827, 0.0015765344509952551, 0.0015224093497742653, 0.0014691928789875575, 0.0014168876555404376, 0.001365496251527869, 0.0013150211941080016,
0.0012654649653778937, 0.0012168300022514433, 0.001169118696339586, 0.0011223333938326484, 0.0010764763953850132, 0.001031549956001947, 0.0009875562849287478, 0.0009444975455420734, 0.0009023758552435935, 0.0008611932853558235, 0.0008209518610203049, 0.0007816535610980103, 0.000743300318072011, 0.0007058940179524842, 0.0006694365001839353, 0.0006339295575547732, 0.0005993749361091205, 0.0005657743350609823, 0.0005331294067106618, 0.0005014417563635255, 0.00047071294225106366, 0.00044094447545424577, 0.0004121378198292347, 0.0003842943919353914, 0.0003574155609656193, 0.0003315026486790318, 0.00030655692933595935, 0.00028257962963528226, 0.0002595719286541187, 0.00023753495778982447, 0.0002164698007043797, 0.00019637749327108933, 0.00017725902352363487, 0.00015911533160750668, 0.0001419473097337609, 0.00012575580213514793, 0.00011054160502459665, 9.630546655606365e-05, 8.30480867877359e-05, 7.077011764761832e-05, 5.947216290146162e-05, 4.915477812307945e-05, 3.9818470667034586e-05, 3.146369964366791e-05, 2.40908758965519e-05, 1.7700361982262436e-05, 1.2292472152559687e-05, 7.867472338942694e-06, 4.425580139555408e-06, 1.9669648084996717e-06, 4.91747247508556e-07, 0.04, 0.03999987706281028, 0.039999508252752496, 0.03999889357436067, 0.039998033035191505, 0.039996926645824216, 0.039995574419860444]

In [ ]:
plt.plot(cosine_annealling_lrs_c)
plt.title("Learning rate per epoch using cosine annealling\nwith T_0=7 and T_mult=2 for 896 epochs")
plt.xlabel("epoch")
plt.ylabel("learning rate")
plt.gcf().set_size_inches(15, 5)
plt.savefig("visualizations/cosine_annealling_learning_rate_896_epochs_T0-7_Tmult-2.png")